# OWL and Blazegraph 
This programme is to practice access to Blazegraph from python. It creates a sample ontology and tries to upload it to BG and then conduct a SPARQL query. 

In [20]:
# %pip install owlready2
# %pip install requests
# %pip install tabulate


# import libraries
from owlready2 import *
from rdflib import Graph, Namespace
from rdflib.namespace import OWL, RDF, RDFS
from SPARQLWrapper import SPARQLWrapper, JSON
from textwrap import dedent
from tabulate import tabulate

import requests


In [ ]:
# Blazegraph endpoint can also write
blazegraph_url = "https://theworldavatar.io/chemistry/blazegraph/namespace/ontocompchem"

def print_gaussian_calculations():
    # Initialize SPARQL endpoint
    sparql = SPARQLWrapper(blazegraph_url)
    
    # Define the SPARQL query
    query = dedent("""
    PREFIX gc: <http://purl.org/gc/>
    PREFIX ontocompchem: <http://www.theworldavatar.com/ontology/ontocompchem/ontocompchem.owl#>
    PREFIX qudt: <http://data.nasa.gov/qudt/owl/qudt#>

    SELECT ?calculation ?program ?version ?method ?basisSet 
           ?energy ?energyUnit ?species ?runDate
    WHERE {
      ?calculation a ontocompchem:GaussianCalculation ;
                   gc:hasProgram ?program ;
                   gc:hasProgramVersion ?version ;
                   gc:hasLevelOfTheory ?method ;
                   gc:hasBasisSet ?basisSet ;
                   gc:hasSystemEnergiesResult/gc:hasElectronicEnergy ?energyValue ;
                   gc:hasUniqueSpecies ?species ;
                   gc:hasRunDate ?runDate .
      
      ?energyValue gc:hasValue ?energy ;
                   gc:hasUnit ?energyUnit .
    }
    ORDER BY DESC(?runDate)
    LIMIT 10
    """)
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        # Execute query
        results = sparql.query().convert()
        bindings = results["results"]["bindings"]
        
        if not bindings:
            print("No Gaussian calculations found in the knowledge base.")
            return
        
        print("\n" + "="*80)
        print("GAUSSIAN CALCULATION RESULTS".center(80))
        print("="*80 + "\n")
        
        for i, result in enumerate(bindings, 1):
            # Extract and clean values
            calc_uri = clean_uri(result.get("calculation", {}).get("value"))
            program = result.get("program", {}).get("value", "N/A")
            version = result.get("version", {}).get("value", "N/A")
            method = clean_uri(result.get("method", {}).get("value"))
            basis = clean_uri(result.get("basisSet", {}).get("value"))
            energy = result.get("energy", {}).get("value", "N/A")
            unit = clean_uri(result.get("energyUnit", {}).get("value"))
            species = clean_uri(result.get("species", {}).get("value"))
            run_date = result.get("runDate", {}).get("value", "N/A")[:10]  # Get just YYYY-MM-DD
            
            # Print formatted output
            print(f"Calculation #{i}".center(80, "-"))
            print(f"Calculation URI: {calc_uri}")
            print(f"Run Date:        {run_date}")
            print(f"Program:         {program} v{version}")
            print(f"Method:          {method}/{basis}")
            print(f"Electronic Energy: {energy} {unit}")
            print(f"Species:         {species}")
            print("-"*80 + "\n")
            
        print(f"Total calculations found: {len(bindings)}")
        print("="*80)
        
    except Exception as e:
        print(f"\nERROR: Failed to execute SPARQL query\n{str(e)}")

def clean_uri(uri):
    """Helper function to clean URIs for display"""
    if not uri:
        return "N/A"
    return uri.split("/")[-1].split("#")[-1] if "/" in uri or "#" in uri else uri

if __name__ == "__main__":
    print_gaussian_calculations()

#     PREFIX gc: <http://purl.org/gc/>
# PREFIX ontocompchem: <http://www.theworldavatar.com/ontology/ontocompchem/ontocompchem.owl#>
# PREFIX qudt: <http://data.nasa.gov/qudt/owl/qudt#>

# SELECT ?GaussianCalculation (COUNT(?p) AS ?numPredicates) ?runDate
# WHERE {
#   ?GaussianCalculation ?p ?o .
#   OPTIONAL { ?GaussianCalculation ontocompchem:hasCalculationTime ?runDate. }
# }
# GROUP BY ?GaussianCalculation ?runDate
# ORDER BY DESC(?runDate)
# LIMIT 100


No Gaussian calculations found in the knowledge base.


In [23]:
# # Blazegraph endpoint - can only read
# blazegraph_url = "http://localhost:3838/blazegraph/namespace/kb/sparql"

# Blazegraph endpoint can also write
blazegraph_url = "https://theworldavatar.io/chemistry/blazegraph/namespace/ontocompchem"


def get_structured_energy_results():
    # Initialize SPARQL endpoint
    sparql = SPARQLWrapper(blazegraph_url)
    
    # Define your SPARQL query
    query = """
PREFIX ontocompchem: <http://www.theworldavatar.com/ontology/ontocompchem/ontocompchem.owl#>
PREFIX qudt: <http://data.nasa.gov/qudt/owl/qudt#>

SELECT  ?energyValue  ?energyType ?energyResult
WHERE {
  
  ?energyResult a ?energyType .
                
}
ORDER BY DESC(?runDate)
LIMIT 10
    """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        # Execute query and get results
        results = sparql.query().convert()
        bindings = results["results"]["bindings"]
        
        if not bindings:
            print("No energy results found in the knowledge base.")
            return
        
        # Prepare structured output
        structured_output = []
        for i, result in enumerate(bindings, 1):
            energy_value = result.get("energyValue", {}).get("value", "N/A")
            energy_type = result.get("energyType", {}).get("value", "N/A")
            energy_result = result.get("energyResult", {}).get("value", "N/A")
            
            # Clean URIs for better readability
            energy_type = energy_type.split("#")[-1] if "#" in energy_type else energy_type.split("/")[-1]
            energy_result = energy_result.split("/")[-1]
            
            # Format numeric values
            try:
                energy_value = f"{float(energy_value):.6f}"
            except (ValueError, TypeError):
                pass
            
            structured_output.append([
                f"Result {i}",
                energy_type,
                energy_value,
                energy_result
            ])
        
        # Print formatted results
        print("\n" + "="*80)
        print("STRUCTURED ENERGY RESULTS".center(80))
        print("="*80)
        
        headers = ["ID", "Energy Type", "Value", "Result URI"]
        print(tabulate(structured_output, headers=headers, tablefmt="grid"))
        
        print("\nAdditional Metadata:")
        print("-"*40)
        print(f"Total results: {len(structured_output)}")
        print(f"Query executed: {sparql.queryString[:100]}...")  # Show first 100 chars of query
        print("="*80)
        
    except Exception as e:
        print(f"\nERROR: Failed to execute SPARQL query\n{str(e)}")

if __name__ == "__main__":
    get_structured_energy_results()



                           STRUCTURED ENERGY RESULTS                            
+-----------+-------------------+---------+--------------------------------------------------------+
| ID        | Energy Type       | Value   | Result URI                                             |
+===========+===================+=========+========================================================+
| Result 1  | SourcePackage     | N/A     | SourcePackage_cbbae3d9-e544-4f32-a0ea-ba7dac661223     |
+-----------+-------------------+---------+--------------------------------------------------------+
| Result 2  | LevelOfTheory     | N/A     | LevelOfTheory_7c83116a-d755-4bc4-9015-52aa92f8ab78     |
+-----------+-------------------+---------+--------------------------------------------------------+
| Result 3  | LevelOfTheory     | N/A     | LevelOfTheory_486d0639-5071-4bc1-b832-b30a50797d80     |
+-----------+-------------------+---------+--------------------------------------------------------+
| Result 

In [ ]:
# # Blazegraph endpoint
# blazegraph_url = "http://localhost:3838/blazegraph/namespace/kb/sparql"
# Blazegraph endpoint can also write
# blazegraph_url = "https://theworldavatar.io/chemistry/blazegraph/namespace/ontocompchem"


# Blazegraph SPARQL Query Endpoint
sparql_endpoint = blazegraph_url

# SPARQL Query  1
sparql_query = """
SELECT ?s ?p ?o
WHERE {
    ?s ?p ?o .
} LIMIT 10
"""

# Send HTTP request
response = requests.get(sparql_endpoint, params={"query": sparql_query, "format": "json"})

# Check if the request was successful
if response.status_code == 200:
    # Parse JSON response
    results = response.json()

    print(results) # {'head': {'vars': ['person', 'organization']}, 'results': {'bindings': []}}

    # # Print results
    # for result in results["results"]["bindings"]:
    #     organization = result["organization"]["value"]
    #     person = result["person"]["value"]
    #     print(f"Organization: {organization}, Person: {person}")
else:
    print(f"Error: {response.status_code} - {response.text}")

